# Notebook 29 — Inference Pipeline: Predicting Opening Performance for New Players

## Purpose

This notebook demonstrates the complete inference pipeline for making opening recommendations to players **not** in our training set.
We use the 1,000 holdout players (reserved in notebook 28) to validate our approach before deploying to production.

**Production Use**: This pipeline will be used on the website to:
- Fetch a player's game history from the Lichess API
    - Note that for the testing in this notebook, we're just getting an untouched holdout player from our local DB.
- Transform their opening statistics into model-ready features
- Generate personalized opening recommendations
- Display predictions with confidence scores

**Development Focus**: We're building **granular, reusable functions** that can be easily adapted from local DB testing to production API integration.

---

## Pipeline Steps

### 1. **Load Holdout Player Data**
- Select a holdout player from the database (never seen during training)
- Extract their complete opening statistics (wins, draws, losses per opening)
- Retrieve player metadata (rating, name, etc.)
- Verify data quality and completeness
- Will use the same player every time this is run, for testing

### 2. **Transform Data for Model Input**
- Calculate raw performance scores: `(wins + 0.5 × draws) / total_games`
- Completely ignore openings not in training set
    - We got rid of those for good reasons, not helpful/unrepresentative/too generic
- Apply hierarchical Bayesian shrinkage toward opening-specific means
- Normalize player rating using training set parameters (z-score)
- Remap database IDs to training IDs (sequential 0-based indices)
- Encode ECO codes into categorical features (letter and number)
- Convert to PyTorch tensors
    - player id (not sure this is needed), opening ids, eco letters and numbers, rating_z

### 3. **Generate Predictions**
- Load trained model and all required artifacts (mappings, normalization params, etc.)
- Feed transformed data through the model
- Generate predicted performance scores for all valid openings
- Separate predictions into: openings player has played vs. new recommendations

### 4. **Analyze and Display Results**
- Compare predictions to actual performance (for openings player has played)
- Rank and display top opening recommendations
- Visualize prediction quality and confidence
- Save predictions for later analysis

---

**Note**: This notebook only performs inference—the model weights remain fixed. We are validating the deployment pipeline, not retraining.

**✓ Resolved**: Opening statistics files (`opening_stats_white.json`, `opening_stats_black.json`) have been created via notebook 30 to support hierarchical Bayesian shrinkage. These files contain opening-specific means calculated from training data.

## Required Model Artifacts

In this notebook (and in production), we will need to load various model artifact files to help in data processing.

I currently believe that we only need very small files (< 1MB total), which is very doable on the client or server.

From the trained model's artifact directory, we'll need:

### **Core Model & Mappings**
- **`best_model.pt`** (8.3 MB) - Trained PyTorch model weights for inference
  - This will live on the HuggingFace space where we host our model, so we don't need to slow down production by loading it in.
- **Note**: Need to make sure we're using a valid player id, if any. It might have to be the next sequential player id that the model's dataset would expect? Not sure, need to investigate.
- **`opening_id_mappings.json`** (96 KB) - Maps database opening IDs → training IDs (0-based sequential)
  - *Why*: Same as above - embeddings require contiguous indices

### **Normalization & Encoding**
- **`rating_normalization.json`** (4 KB) - Contains `rating_mean` and `rating_std` from training
  - *Why*: Must normalize new player ratings with exact same parameters: `z = (rating - mean) / std`
- **`eco_encodings.json`** (4 KB) - Maps ECO codes to integers (letter: A-E → 0-4, numbers → sequential ints)
  - *Why*: Model expects categorical integers, not raw ECO strings like "B02"

### **Model Configuration**
- **`hyperparameters.json`** (4 KB) - NUM_FACTORS (40), NUM_PLAYERS, NUM_OPENINGS, embedding dimensions
  - *Why*: Need exact architecture specs to instantiate the model correctly
- **`model_architecture.json`** (4 KB) - Complete model structure details
  - *Why*: Documents the model class and layer configurations

### **Reference Data**
- **`opening_mappings.csv`** (168 KB) - Full opening metadata (db_id, training_id, eco, name)
  - *Why*: Look up opening details when displaying recommendations
- **`holdout_players.csv`** (24 KB) - List of 1,000 holdout player IDs for testing
  - *Why*: Select test players that were never seen during training

### **✓ Opening Statistics for Bayesian Shrinkage**
- **`opening_stats_white.json`** (~50-100 KB) - White opening statistics (compact format)
- **`opening_stats_black.json`** (~50-100 KB) - Black opening statistics (compact format)
- **`opening_stats_white.csv`** (~200-300 KB) - Human-readable CSV with full columns
- **`opening_stats_black.csv`** (~200-300 KB) - Human-readable CSV with full columns
  - *Why*: Required for hierarchical Bayesian shrinkage during inference
  - *Status*: **✓ Created via notebook 30** (`30_create_opening_stats.ipynb`)
  - *Structure*: JSON uses compact array format `{training_id: [opening_mean, total_games, db_id]}`
    - Index 0: `opening_mean` - Mean score for Bayesian shrinkage (float)
    - Index 1: `total_games` - Total games across all players (int)
    - Index 2: `db_id` - Database opening ID for reference (int)
  - *Usage*: Load at startup, use `opening_stats[training_id][0]` for shrinkage target
  - *CSV*: For manual inspection, contains full columns (training_id, db_id, eco, name, opening_mean, opening_total_games, opening_num_players)

---

**Total artifact size**: ~1MB for all files combined.

## Step 1: Load Holdout Player Data

We'll:
1. Load the holdout players list from model artifacts
2. Select a deterministic player (using fixed index from specs)
3. Verify they're NOT in the training set
4. Extract their complete opening statistics from database (for specified color)
5. Retrieve their player metadata
6. Display everything for verification

**Deterministic Selection**: Using `HOLDOUT_PLAYER_INDEX` from specs for consistency across runs.

**Color**: Extracting data for the color specified in pipeline specs (White='w' or Black='b').

## Pipeline Specifications

Adjustable parameters for the inference pipeline.

In [1]:
# ============================================================================
# PIPELINE SPECIFICATIONS - Adjust these parameters as needed
# ============================================================================

# Color to analyze ('w' for White, 'b' for Black)
COLOR = 'w'

# Deterministic player selection (which holdout player to use)
HOLDOUT_PLAYER_INDEX = 0

# Model artifacts directory name
MODEL_DIR_NAME = "20251212_152017_black"

# ============================================================================

# Derived display values
COLOR_NAME = "White" if COLOR == 'w' else "Black"

print(" Pipeline Specifications:")
print(f"   Color: {COLOR_NAME} ('{COLOR}')")
print(f"   Holdout player index: {HOLDOUT_PLAYER_INDEX}")
print(f"   Model directory: {MODEL_DIR_NAME}")
print("="*80)

 Pipeline Specifications:
   Color: White ('w')
   Holdout player index: 0
   Model directory: 20251212_152017_black


In [2]:
# Standard library imports
import sys
from pathlib import Path
import json

# Third-party imports
import pandas as pd
import numpy as np

# Add project root to path for module imports
PROJECT_ROOT = Path.cwd().parent
sys.path.insert(0, str(PROJECT_ROOT))

from utils.database.db_utils import get_db_connection

print("✓ All imports successful")

✓ All imports successful


In [3]:
# Configuration - paths constructed from specifications
DB_PATH = PROJECT_ROOT / "data" / "processed" / "chess_games.db"
MODEL_ARTIFACTS_DIR = PROJECT_ROOT / "data" / "models" / MODEL_DIR_NAME

# Verify paths exist
print(" Verifying paths...")
print(f"   Database: {DB_PATH.exists()} - {DB_PATH}")
print(f"   Model artifacts: {MODEL_ARTIFACTS_DIR.exists()} - {MODEL_ARTIFACTS_DIR}")

if not DB_PATH.exists():
    raise FileNotFoundError(f"Database not found at {DB_PATH}")
if not MODEL_ARTIFACTS_DIR.exists():
    raise FileNotFoundError(f"Model artifacts not found at {MODEL_ARTIFACTS_DIR}")

print("✓ All paths verified")

 Verifying paths...
   Database: True - /Users/a/Documents/personalprojects/chess-opening-recommender/data/processed/chess_games.db
   Model artifacts: True - /Users/a/Documents/personalprojects/chess-opening-recommender/data/models/20251212_152017_black
✓ All paths verified


In [4]:
# Step 1.1: Load holdout players list
print("Loading holdout players...")

holdout_players_path = MODEL_ARTIFACTS_DIR / "holdout_players.csv"
holdout_players_df = pd.read_csv(holdout_players_path)

print(f"   Total holdout players: {len(holdout_players_df):,}")
print(f"   Columns: {list(holdout_players_df.columns)}")
print(f"\n   First 5 holdout players:")
print(holdout_players_df.head())

# Select our test player deterministically (always use the first player)
test_player_db_id = int(holdout_players_df.iloc[HOLDOUT_PLAYER_INDEX]['db_id'])
test_player_name = holdout_players_df.iloc[HOLDOUT_PLAYER_INDEX]['name']
test_player_rating = int(holdout_players_df.iloc[HOLDOUT_PLAYER_INDEX]['rating'])

print(f"\n✓ Selected holdout player #{HOLDOUT_PLAYER_INDEX}:")
print(f"   Database ID: {test_player_db_id}")
print(f"   Name: {test_player_name}")
print(f"   Rating: {test_player_rating}")

Loading holdout players...
   Total holdout players: 1,000
   Columns: ['db_id', 'name', 'title', 'rating']

   First 5 holdout players:
   db_id        name  title  rating
0     53    AAashraf    NaN    1723
1    129       AK988    NaN    1489
2    187  ALexxxey89    NaN    1635
3    425   Abgrenzer    NaN    1787
4    498      Ac1000    NaN    2541

✓ Selected holdout player #0:
   Database ID: 53
   Name: AAashraf
   Rating: 1723


In [5]:
# Step 1.2: Verify player is NOT in training set
print("Verifying player is not in training set...")

player_mappings_path = MODEL_ARTIFACTS_DIR / "player_mappings.csv"
player_mappings_df = pd.read_csv(player_mappings_path)

print(f"   Total training players: {len(player_mappings_df):,}")
print(f"   Columns: {list(player_mappings_df.columns)}")

# Check if our test player is in the training set
is_in_training = test_player_db_id in player_mappings_df['db_id'].values

if is_in_training:
    raise ValueError(
        f"❌ ERROR: Player {test_player_name} (ID: {test_player_db_id}) "
        f"is in the training set! They should be a holdout player."
    )


Verifying player is not in training set...
   Total training players: 48,551
   Columns: ['db_id', 'name', 'title', 'rating', 'training_id']


In [6]:
# Step 1.3: Extract player's opening statistics and create PlayerData object
print(f" Extracting {COLOR_NAME} opening statistics for {test_player_name}...")

from utils.foldin_data_processing.types import PlayerData

conn = get_db_connection(DB_PATH)

# Query returns data matching OpeningStatsRow schema
query = """
SELECT 
    pos.player_id,
    pos.opening_id,
    o.eco,
    o.name as opening_name,
    pos.num_wins + pos.num_draws + pos.num_losses as num_games,
    pos.num_wins,
    pos.num_draws,
    pos.num_losses
FROM player_opening_stats pos
JOIN opening o ON pos.opening_id = o.id
WHERE pos.player_id = ?
  AND pos.color = ?
  AND (pos.num_wins + pos.num_draws + pos.num_losses) > 0
ORDER BY (pos.num_wins + pos.num_draws + pos.num_losses) DESC
"""

# Execute query and get DataFrame conforming to OpeningStatsRow schema
opening_stats_df = conn.execute(query, [int(test_player_db_id), COLOR]).df()
conn.close()

# Immediately wrap in PlayerData object
player_data = PlayerData(
    player_id=test_player_db_id,
    name=test_player_name,
    rating=test_player_rating,
    color=COLOR,
    opening_stats_df=opening_stats_df
)

print(f"✓ Database query complete")
print(f"   Total openings played: {len(player_data.opening_stats_df)}")
print(f"   Total games: {player_data.total_games():,}")


 Extracting White opening statistics for AAashraf...
✓ Database query complete
   Total openings played: 192
   Total games: 3,050


In [7]:
# Step 1.4: Display complete player data for verification
print("="*80)
print(f"DATA FOR TEST PLAYER: {test_player_name} ({COLOR_NAME})")
print("="*80)

print("\n PLAYER METADATA:")
print(f"   Database ID: {test_player_db_id}")
print(f"   Name: {test_player_name}")
print(f"   Rating: {test_player_rating}")
print(f"   Color: {COLOR_NAME}")
print(f"   Status: HOLDOUT PLAYER (not in training set)")

print(f"\n {COLOR_NAME.upper()} OPENING STATISTICS SUMMARY:")
print(f"   Total openings played: {len(player_data.opening_stats_df)}")
print(f"   Total games: {player_data.total_games():,}")
print(f"   Total wins: {player_data.total_wins()}")
print(f"   Total draws: {player_data.opening_stats_df['num_draws'].sum()}")
print(f"   Total losses: {player_data.opening_stats_df['num_losses'].sum()}")
print(f"   Overall win rate: {player_data.total_wins() / player_data.total_games():.1%}")

print(f"\n TOP 5 MOST-PLAYED {COLOR_NAME.upper()} OPENINGS:")
print(player_data.opening_stats_df[['opening_name', 'eco', 'num_games', 'num_wins', 'num_draws', 'num_losses']].head(5).to_string(index=False))

print(f"\n {COLOR_NAME} opening data extracted successfully for {test_player_name}")
print(f"   Ready to proceed to data transformation (Step 2)")
print("="*80)

DATA FOR TEST PLAYER: AAashraf (White)

 PLAYER METADATA:
   Database ID: 53
   Name: AAashraf
   Rating: 1723
   Color: White
   Status: HOLDOUT PLAYER (not in training set)

 WHITE OPENING STATISTICS SUMMARY:
   Total openings played: 192
   Total games: 3,050
   Total wins: 1474
   Total draws: 259
   Total losses: 1317
   Overall win rate: 48.3%

 TOP 5 MOST-PLAYED WHITE OPENINGS:
                      opening_name eco  num_games  num_wins  num_draws  num_losses
                 Caro-Kann Defense B10        248       123         19         106
  French Defense: Knight Variation C00        201        96         13          92
                  Philidor Defense C41        145        76         18          51
  Sicilian Defense: Bowdler Attack B20        104        48          9          47
Sicilian Defense: French Variation B40        104        45         11          48

 White opening data extracted successfully for AAashraf
   Ready to proceed to data transformation (Step 2)


---

# STEP 2: Transform Player Data for Model Inference

This is the HEART of the production system. Every time a player visits the website, I'll:
1. Download their games from Lichess API
2. Process them through the pipeline below
3. Send the result to HuggingFace for fold-in inference

---

## Objective

Transform raw player opening statistics into model-ready tensors that can be sent directly to HuggingFace.

**Key Principle**: This transformation must be IDENTICAL whether data comes from:
- Local DB (for notebook testing)
- Lichess API (for production)

---

## Architecture Overview

```
PlayerData (from Step 1)
    |
    v
[2.1] Define Type System
    |
    v
[2.2] Load Model Artifacts (once at startup)
    |
    v
[2.3] Filter & Validate Openings
    |
    v
[2.4] Calculate Raw Scores
    |
    v
[2.5] Apply Bayesian Shrinkage
    |
    v
[2.6] Normalize Rating
    |
    v
[2.7] Remap Database IDs -> Training IDs
    |
    v
[2.8] Encode ECO Features
    |
    v
[2.9] Convert to NumPy Arrays
    |
    v
ModelInput -> HuggingFace API
```

---

## Implementation Steps

### 2.1: Define Type System

#### ACCOMPLISHED: Defined these types and stored them in notebooks/utils/foldin_data_processing/types.py

**Goal**: Create strongly-typed dataclasses for all data structures

**What I'll build**:
- `RawOpeningStats`: Input from Step 1 (database/API)
- `PlayerData`: Complete player profile
- `ProcessedOpening`: After transformations
- `ModelInput`: Final structure for HuggingFace

**Why this matters**:
- Catch type errors at development time
- Self-documenting code
- IDE autocomplete and type checking
- Makes data flow crystal clear

**Outputs**: Python dataclasses with type hints and docstrings

---

### 2.2: Load Model Artifacts

**Goal**: Load all training artifacts into memory for fast lookup

**Note that I will revisit this once I'm more sure what I need to load and when**

**Required artifacts** (from training notebook 28):
1. `opening_mappings.csv` - Database IDs to Training IDs
2. `rating_normalization.json` - Rating mean/std
3. `eco_encodings.json` - ECO letter/number mappings
4. `opening_stats.json` - NEED TO CREATE THIS

**Data structures to build**:
- `valid_training_ids: Set[int]` - O(1) lookup for filtering
- `db_to_training_id: Dict[int, int]` - O(1) ID remapping
- `opening_means: Dict[int, float]` - Bayesian shrinkage targets
- `eco_letter_map: Dict[str, int]` - ECO letter encoding
- `eco_number_map: Dict[str, int]` - ECO number encoding

**Critical**: Load ONCE at startup, cache in production

**Outputs**: Dictionary of all artifacts plus fast lookup structures

---

### 2.3: Filter & Validate Openings

**Goal**: Remove openings not in training set or below threshold

**NOTE**: This is originally being written as a proof-of-concept, tested against a holdout player who was part of our training db (but the model wasn't trained on their games). So, in development, this should filter out few or no openings. But in dev, we'll be getting players directly from lichess so they will have openings filtered out.

**Filtering criteria**:
1. Opening must be in training set (check against `valid_training_ids`)
2. Player must have played >= `MIN_GAMES_THRESHOLD` with this opening (default: 10)

**Edge case handling**:
- What if player has 0 valid openings after filtering? -> Raise `ValueError`
- What if opening ID not in mapping? -> Skip silently (expected)

**Efficiency**: Use Set for O(1) membership testing

**Outputs**: Filtered list of `RawOpeningStats`

---

### 2.4: Calculate Raw Scores

**Goal**: Convert W/D/L to performance scores

**Formula**: 
```
raw_score = (wins + 0.5 * draws) / total_games
```

**Validation**:
- Score must be in range [0.0, 1.0]
- Handle edge case where `num_games = 0` (shouldn't happen after filtering)

**Outputs**: Add `raw_score` property to each opening

---

### 2.5: Apply Bayesian Shrinkage

**Goal**: Adjust scores toward opening-specific means (hierarchical Bayesian)

**Formula**:
```
adjusted_score = (num_games * raw_score + k * opening_mean) / (num_games + k)
confidence = num_games / (num_games + k)
```

**Constants** (must match training):
- `k = 50` (shrinkage strength)
- `opening_mean` from `opening_stats.json` (keyed by training_id, not db_id!)

**Why this matters**:
- Prevents overfitting to small sample sizes
- Makes inference consistent with training
- Different openings have different baseline win rates

**Critical**: Use opening-specific means, NOT global mean

**Outputs**: `ProcessedOpening` objects with `adjusted_score` and `confidence`

---

#### 📊 Opening Stats Files Documentation

The opening statistics files were created in **notebook 30** (`30_create_opening_stats.ipynb`) to support hierarchical Bayesian shrinkage during inference.

**Files created for each color**:
- `opening_stats_{color}.json` - Compact format for production (~50-100 KB per color)
- `opening_stats_{color}.csv` - Human-readable format for inspection (~200-300 KB per color)

**JSON Structure (Compact Array Format)**:
```json
{
  "training_id": [opening_mean, total_games, db_id],
  "0": [0.5234, 15847, 123],
  "1": [0.4891, 8932, 456],
  ...
}
```

**Array indices**:
- `[0]`: `opening_mean` (float) - Mean score for this opening across all training players. Use this for Bayesian shrinkage.
- `[1]`: `total_games` (int) - Total games played with this opening across all players.
- `[2]`: `db_id` (int) - Original database opening ID (for reference/debugging).

**Usage in Step 2.5**:
```python
# Load at startup (once)
with open(f'opening_stats_{color}.json', 'r') as f:
    opening_stats = json.load(f)

# During shrinkage (per opening)
training_id = db_to_training_id[opening_db_id]
opening_mean = opening_stats[str(training_id)][0]  # JSON keys are strings
adjusted_score = (num_games * raw_score + k * opening_mean) / (num_games + k)
```

**CSV Structure** (for manual inspection):
- Columns: `training_id`, `db_id`, `opening_id`, `eco`, `name`, `opening_mean`, `opening_total_games`, `opening_num_players`
- Sorted by training_id for easy lookup
- Contains full opening names and ECO codes for human readability

**Data Source**:
- Aggregated from `player_opening_stats` table in training database
- Filtered to openings with ≥10 games per player (same threshold as training)
- Calculated as: `mean((wins + 0.5 * draws) / total_games)` across all players

**Why compact format?**:
- Original verbose dict format was ~500 KB (too large for production)
- Compact array format reduces size to ~50-100 KB per color
- Faster to load and parse in production environment
- Keys are training IDs (not db IDs) for O(1) lookup during inference

---

### 2.6: Normalize Rating

**Goal**: Z-score normalize player rating

**Formula**:
```
rating_z = (rating - RATING_MEAN) / RATING_STD
```

**Constants from artifacts**:
- `RATING_MEAN` and `RATING_STD` from `rating_normalization.json`

**Why this matters**:
- Model was trained on normalized ratings
- Keeps all features on similar scales

**Outputs**: Single float `rating_z`

---

### 2.7: Remap Opening IDs

**Goal**: Convert database IDs to training IDs (0-based sequential)

**Why this is necessary**:
- Database IDs are arbitrary (10, 15, 23, ...)
- Model embedding layers expect 0-based sequential IDs (0, 1, 2, ...)
- Training created this mapping during preprocessing

**Lookup**: Use `db_to_training_id` dictionary (O(1))

**Validation**: Ensure all remapped IDs are in valid range [0, num_openings)

**Outputs**: Update `training_opening_id` in each `ProcessedOpening`

---

### 2.8: Encode ECO Features

**Goal**: Convert ECO strings to categorical integers

**ECO structure**: "C21" -> letter ("C") + number ("21")

**Encoding**:
- `eco_letter_cat`: "A"->0, "B"->1, "C"->2, "D"->3, "E"->4
- `eco_number_cat`: "00"->0, "01"->1, ..., "99"->99 (sequential)

**Edge case**: What if ECO not in mapping? -> Log warning, use default value

**Why this matters**:
- Model learns patterns by ECO category (e.g., all "C" openings share features)
- Categorical encoding is more efficient than one-hot

**Outputs**: Add `eco_letter_cat` and `eco_number_cat` to each `ProcessedOpening`

---

### 2.9: Build ModelInput Structure

**Goal**: Convert processed openings to NumPy arrays for HuggingFace

**Output structure**:
```python
ModelInput(
    player_name: str,
    rating_z: float,
    color: str,
    opening_ids: np.ndarray,      # shape (N,), dtype int64
    eco_letter_cats: np.ndarray,  # shape (N,), dtype int64
    eco_number_cats: np.ndarray,  # shape (N,), dtype int64
    scores: np.ndarray,           # shape (N,), dtype float32
    confidence: np.ndarray,       # shape (N,), dtype float32
    opening_names: List[str],     # for display
    num_openings_filtered: int,   # metadata
    total_games: int              # metadata
)
```

**Methods**:
- `to_json_dict()` -> JSON for HuggingFace API
- `to_tensors()` -> PyTorch tensors for local testing

**Why NumPy not PyTorch**: 
- NumPy serializes to JSON easily
- HuggingFace endpoint will convert to tensors on its side

**Outputs**: Ready-to-send `ModelInput` object

---

### 2.10: Main Transformation Function

**Goal**: Orchestrate all chunks into one reusable function

**Signature**:
```python
def transform_player_for_inference(
    player_data: PlayerData,
    model_artifacts: Dict,
    min_games_threshold: int = 10,
    k_shrinkage: int = 50
) -> ModelInput:
    """Production-ready transformation function."""
```

**This function is the PUBLIC API** - it's what will be called in production

**Error handling**:
- Raise `ValueError` if no valid openings
- Raise `FileNotFoundError` if artifacts missing
- Log warnings for edge cases

**Outputs**: Single `ModelInput` object ready for inference

---

### 2.11: Test & Validate

**Goal**: Verify transformation on holdout player

**Tests**:
1. Run transformation on test player from Step 1
2. Check ID ranges (0 <= opening_id < num_openings)
3. Verify scores in [0, 1]
4. Confirm ECO encodings are valid
5. Validate rating normalization
6. Check JSON serialization works
7. Compare with training data format

**Edge cases to test**:
- Player with many openings
- Player with few openings
- Player near MIN_GAMES_THRESHOLD
- High/low rated players

**Outputs**: Validated `ModelInput` plus test results

---

## Critical Notes

### ID Systems - PAY ATTENTION

- **Database IDs**: What's in the DB and Lichess API (arbitrary integers)
- **Training IDs**: 0-based sequential IDs used by model (0, 1, 2, ...)
- **Note:** When I originally trained the model, I tried to normalize these two, mutating the DB ids to match the new training IDs. But, it turned out that was really hard so I stopped trying.
- **Transformation happens in 2.7**

### Bayesian Shrinkage - MUST MATCH TRAINING

- Training used opening-specific means (hierarchical)
- Inference MUST use the same approach
- Requires `opening_stats.json` (keyed by training_id)

### Production vs Testing

- Testing: Data from local DB (Step 1)
- Production: Data from Lichess API (future code)
- Transformation is IDENTICAL - that's the whole point

### Performance Optimization

- Load artifacts ONCE at startup
- Use Sets for O(1) membership testing
- Use Dicts for O(1) ID remapping
- Vectorize NumPy operations (no loops)

---

## Before Starting Coding Checklist

- [ ] Do I have all 4 artifact files?
  - [ ] `opening_mappings.csv`
  - [ ] `rating_normalization.json`
  - [ ] `eco_encodings.json`
  - [ ] `opening_stats.json` (NEED TO CREATE)

- [ ] Have I reviewed the training preprocessing to match exactly?
- [ ] Do I understand the ID remapping workflow?
- [ ] Am I clear on hierarchical vs global Bayesian shrinkage?

---

## Success Criteria

Step 2 is complete when:
1. All type definitions are complete and well-documented
2. All artifacts load successfully
3. Transformation function runs without errors
4. Output structure matches HuggingFace API expectations
5. All validation checks pass
6. Edge cases are handled gracefully
7. Code is production-ready and reusable

---

Ready to implement.

In [8]:
# 2.1 Define Tpes, Classes and Data Structures

# We've done that in notebooks/utils/foldin_data_processing/types.py

In [9]:
# 2.2 Load model artifacts
# Placeholder: Will flesh this out more once we're sure what we need and when

# Load artefacts here, not in individual cells below

opening_mappings_df = pd.read_csv(MODEL_ARTIFACTS_DIR / "opening_mappings.csv")
valid_training_opening_ids = set(opening_mappings_df["db_id"].values)

In [10]:
# 2.3 Filter and Validate Openings

# I axed certain openings from the training data because they were unrepresentative, had weird scores, low sample size, etc

# Resultingly, this pipeline will completely ignore those openings when making recommendations. This cell filters them out.

# NOTE: This function is originally being tested as a proof-of-concept against a holdout player who is extracted from a local db where the openings in question have already been filtered out.
    # So you won't see any openings dropped here when you run it against that holdout player.
# However, in prod, when we run against real users whose data is not pre-filtered, we should see openings dropped here.


def filter_valid_openings(
    player_data: PlayerData,
    valid_opening_local_db_ids: set[int],
    min_num_games_threshold: int = 10,
) -> PlayerData:
    """
    Filter player's opening statistics to only include openings
    that exist in the training set and meet minimum game threshold.

    Args:
        player_data: Complete player profile from Step 1
        valid_opening_local_db_ids: Set of database opening IDs from training
        min_num_games_threshold: Minimum games required per opening (default: 10)

    Returns:
        New PlayerData object with filtered opening_stats_df

    Raises:
        ValueError: If no valid openings remain after filtering

    Example:
        >>> filtered_data = filter_valid_openings(player_data, valid_ids, min_num_games_threshold=10)
        >>> print(f"Kept {len(filtered_data.opening_stats_df)} / {len(player_data.opening_stats_df)} openings")
    """
    print(f"\n{'='*80}")
    print("STEP 2.3: Filtering Valid Openings")
    print(f"{'='*80}")

    original_df = player_data.opening_stats_df
    print(f"\nOriginal data:")
    print(f"   Total openings: {len(original_df)}")
    print(f"   Total games: {original_df['num_games'].sum():,}")
    print(f"   Training set size: {len(valid_opening_local_db_ids):,} openings")

    # Filter 1: Opening must be in training set (O(1) lookup with set)
    print(f"\nFilter 1: Must be in training set...")
    in_training_mask = original_df["opening_id"].isin(valid_opening_local_db_ids)
    after_training_filter = original_df[in_training_mask].copy()

    dropped_not_in_training = len(original_df) - len(after_training_filter)
    print(f"   Kept: {len(after_training_filter)} openings")
    print(f"   Dropped: {dropped_not_in_training} openings (not in training set)")

    if dropped_not_in_training > 0:
        dropped_games = original_df[~in_training_mask]["num_games"].sum()
        print(
            f"   Games in dropped openings: {dropped_games:,} ({dropped_games / original_df['num_games'].sum():.1%} of total)"
        )

    # Filter 2: Must have minimum games
    print(f"\nFilter 2: Must have >= {min_num_games_threshold} games...")
    games_mask = after_training_filter["num_games"] >= min_num_games_threshold
    filtered_df = after_training_filter[games_mask].copy()

    dropped_low_games = len(after_training_filter) - len(filtered_df)
    print(f"   Kept: {len(filtered_df)} openings")
    print(f"   Dropped: {dropped_low_games} openings (< {min_num_games_threshold} games)")

    if dropped_low_games > 0:
        dropped_games = after_training_filter[~games_mask]["num_games"].sum()
        print(f"   Games in dropped openings: {dropped_games:,}")

    # Validation: Must have at least one opening remaining
    if len(filtered_df) == 0:
        total_dropped = dropped_not_in_training + dropped_low_games
        raise ValueError(
            f"No valid openings remaining for player {player_data.name}!\n"
            f"   Original openings: {len(original_df)}\n"
            f"   Dropped (not in training): {dropped_not_in_training}\n"
            f"   Dropped (< {min_num_games_threshold} games): {dropped_low_games}\n"
            f"   Remaining: 0\n"
            f"Cannot make recommendations for this player."
        )

    # Create new PlayerData with filtered DataFrame
    filtered_player_data = PlayerData(
        player_id=player_data.player_id,
        name=player_data.name,
        rating=player_data.rating,
        color=player_data.color,
        opening_stats_df=filtered_df,
    )

    # Summary statistics
    print(f"\n✓ Filtering complete:")
    print(
        f"   Final openings: {len(filtered_df)} ({len(filtered_df) / len(original_df):.1%} of original)"
    )
    print(
        f"   Final games: {filtered_df['num_games'].sum():,} ({filtered_df['num_games'].sum() / original_df['num_games'].sum():.1%} of original)"
    )
    print(f"   Total dropped: {dropped_not_in_training + dropped_low_games} openings")

    # Show sample of kept openings
    print(f"\n   Top 5 kept openings (by games):")
    top_5 = filtered_df.nlargest(5, "num_games")[["opening_name", "eco", "num_games"]]
    for idx, row in top_5.iterrows():
        print(
            f"      {row['eco']:>3} | {row['opening_name']:<50} | {row['num_games']:>4} games"
        )

    print("=" * 80)

    return filtered_player_data

In [11]:
player_data = filter_valid_openings(
    player_data,
    valid_opening_local_db_ids=valid_training_opening_ids,
    min_num_games_threshold=3,
)


STEP 2.3: Filtering Valid Openings

Original data:
   Total openings: 192
   Total games: 3,050
   Training set size: 2,728 openings

Filter 1: Must be in training set...
   Kept: 192 openings
   Dropped: 0 openings (not in training set)

Filter 2: Must have >= 3 games...
   Kept: 108 openings
   Dropped: 84 openings (< 3 games)
   Games in dropped openings: 112

✓ Filtering complete:
   Final openings: 108 (56.2% of original)
   Final games: 2,938 (96.3% of original)
   Total dropped: 84 openings

   Top 5 kept openings (by games):
      B10 | Caro-Kann Defense                                  |  248 games
      C00 | French Defense: Knight Variation                   |  201 games
      C41 | Philidor Defense                                   |  145 games
      B20 | Sicilian Defense: Bowdler Attack                   |  104 games
      B40 | Sicilian Defense: French Variation                 |  104 games


In [12]:
# Step 2.4: Calculate Raw Scores (vectorized, production-ready)

import numpy as np
import pandas as pd

def add_raw_scores(player_data: PlayerData) -> PlayerData:
    """
    Add raw performance score per opening:
        raw_score = (wins + 0.5 * draws) / num_games

    Assumes player_data.opening_stats_df contains:
      - num_wins, num_draws, num_losses, num_games

    Returns a NEW PlayerData with an additional 'raw_score' float column.
    """
    print(f"\n{'='*80}")
    print("STEP 2.4: Calculating Raw Scores")
    print(f"{'='*80}")

    df = player_data.opening_stats_df.copy()

    required_cols = {"num_wins", "num_draws", "num_losses", "num_games"}
    missing = required_cols - set(df.columns)
    if missing:
        raise ValueError(f"opening_stats_df missing required columns: {sorted(missing)}")

    # Sanity checks
    if (df["num_games"] < 0).any():
        raise ValueError("Found negative num_games values (invalid data).")

    # Enforce consistency if num_games is present (it is in your query)
    computed_games = df["num_wins"] + df["num_draws"] + df["num_losses"]
    inconsistent = (computed_games != df["num_games"])
    if inconsistent.any():
        bad = df.loc[inconsistent, ["opening_id", "num_wins", "num_draws", "num_losses", "num_games"]].head(10)
        raise ValueError(
            "num_games is inconsistent with wins+draws+losses for some rows. "
            "Sample:\n" + bad.to_string(index=False)
        )

    # Vectorized score calculation; guard against divide-by-zero
    denom = df["num_games"].replace({0: np.nan})
    df["raw_score"] = (df["num_wins"] + 0.5 * df["num_draws"]) / denom
    df["raw_score"] = df["raw_score"].fillna(0.0).astype(float)

    # Range validation (allow tiny floating error)
    min_score = float(df["raw_score"].min())
    max_score = float(df["raw_score"].max())
    if min_score < -1e-9 or max_score > 1.0 + 1e-9:
        bad = df.loc[(df["raw_score"] < 0) | (df["raw_score"] > 1), ["opening_id", "num_games", "num_wins", "num_draws", "raw_score"]].head(10)
        raise ValueError(
            f"raw_score out of range [0,1]. Observed range [{min_score:.6f}, {max_score:.6f}]. "
            "Sample offending rows:\n" + bad.to_string(index=False)
        )

    # Logging
    print(f"\n✓ Raw scores computed")
    print(f"   Openings: {len(df):,}")
    print(f"   Total games: {int(df['num_games'].sum()):,}")
    print(f"   raw_score range: [{min_score:.4f}, {max_score:.4f}]")
    print(f"   raw_score mean:  {float(df['raw_score'].mean()):.4f}")

    # Show the top 5 by volume with their raw scores
    preview_cols = ["opening_name", "eco", "num_games", "num_wins", "num_draws", "num_losses", "raw_score"]
    top5 = df.nlargest(5, "num_games")[preview_cols]
    print(f"\nTop 5 most-played openings with raw_score:")
    print(top5.to_string(index=False))

    print("=" * 80)

    return PlayerData(
        player_id=player_data.player_id,
        name=player_data.name,
        rating=player_data.rating,
        color=player_data.color,
        opening_stats_df=df,
    )



# Then add raw scores
player_data = add_raw_scores(player_data)

# Display the result
player_data.opening_stats_df.head()


STEP 2.4: Calculating Raw Scores

✓ Raw scores computed
   Openings: 108
   Total games: 2,938
   raw_score range: [0.0000, 1.0000]
   raw_score mean:  0.5192

Top 5 most-played openings with raw_score:
                      opening_name eco  num_games  num_wins  num_draws  num_losses  raw_score
                 Caro-Kann Defense B10        248       123         19         106   0.534274
  French Defense: Knight Variation C00        201        96         13          92   0.509950
                  Philidor Defense C41        145        76         18          51   0.586207
  Sicilian Defense: Bowdler Attack B20        104        48          9          47   0.504808
Sicilian Defense: French Variation B40        104        45         11          48   0.485577


,player_id,opening_id,eco,opening_name,num_games,num_wins,num_draws,num_losses,raw_score
0,53,910,B10,Caro-Kann Defense,248,123,19,106,0.534274
1,53,1356,C00,French Defense: Knight Variation,201,96,13,92,0.509950
2,53,1854,C41,Philidor Defense,145,76,18,51,0.586207
3,53,1001,B20,Sicilian Defense: Bowdler Attack,104,48,9,47,0.504808
4,53,1167,B40,Sicilian Defense: French Variation,104,45,11,48,0.485577
